https://docs.sqlalchemy.org/en/13/orm/tutorial.html

In [67]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [68]:
import sqlalchemy
sqlalchemy.__version__ 

from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

engine

from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

Base

'1.3.1'

Engine(sqlite:///:memory:)

sqlalchemy.ext.declarative.api.Base

In [69]:
from sqlalchemy import Column, Integer, String
class User(Base):
     __tablename__ = 'users'

     id = Column(Integer, primary_key=True)
     name = Column(String)
     fullname = Column(String)
     nickname = Column(String)

     def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
                             self.name, self.fullname, self.nickname)

In [70]:
User.__table__ 

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [71]:
Base.metadata.create_all(engine)

2019-04-18 20:19:32,718 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-04-18 20:19:32,719 INFO sqlalchemy.engine.base.Engine ()
2019-04-18 20:19:32,721 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-04-18 20:19:32,722 INFO sqlalchemy.engine.base.Engine ()
2019-04-18 20:19:32,723 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-04-18 20:19:32,724 INFO sqlalchemy.engine.base.Engine ()
2019-04-18 20:19:32,726 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2019-04-18 20:19:32,727 INFO sqlalchemy.engine.base.Engine ()
2019-04-18 20:19:32,731 INFO sqlalchemy.engine.base.Engine COMMIT


In [72]:
# from sqlalchemy import Sequence
# Column(Integer, Sequence('user_id_seq'), primary_key=True)

In [73]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
ed_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [74]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine, autoflush=False)

In [75]:
session = Session()
session.add(ed_user)

In [76]:
ed_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [77]:
session.commit()

2019-04-18 20:19:36,763 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-18 20:19:36,766 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-04-18 20:19:36,767 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2019-04-18 20:19:36,769 INFO sqlalchemy.engine.base.Engine COMMIT


In [78]:
session.query(User).filter_by(name='ed').first()

2019-04-18 20:19:41,273 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-04-18 20:19:41,275 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2019-04-18 20:19:41,276 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [79]:
our_user = session.query(User).filter_by(name='ed').first() 
our_user

2019-04-18 20:19:49,893 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2019-04-18 20:19:49,895 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [80]:
our_user

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [81]:
ed_user is our_user
ed_user
our_user

True

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>

In [82]:
session.add_all([
     User(name='wendy', fullname='Wendy Williams', nickname='windy'),
     User(name='mary', fullname='Mary Contrary', nickname='mary'),
     User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [83]:
ed_user.nickname = 'eddie'

In [84]:
session.dirty

IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])

In [85]:
session.new

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [86]:
session.commit()

2019-04-18 20:20:19,792 INFO sqlalchemy.engine.base.Engine UPDATE users SET nickname=? WHERE users.id = ?
2019-04-18 20:20:19,794 INFO sqlalchemy.engine.base.Engine ('eddie', 1)
2019-04-18 20:20:19,795 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-04-18 20:20:19,796 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2019-04-18 20:20:19,797 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-04-18 20:20:19,798 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2019-04-18 20:20:19,799 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-04-18 20:20:19,799 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2019-04-18 20:20:19,800 INFO sqlalchemy.engine.base.Engine COMMIT
